In [4]:
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from tqdm import tqdm
from wordcloud import WordCloud
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import twint
import nest_asyncio

nest_asyncio.apply()

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Raizel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Raizel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Raizel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+')

In [8]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

# Inception

In [9]:
df_1 = pd.read_csv('df_1.csv')
df_1.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,30776953027956739,30776953027956739,1.296173e+12,2011-01-27 18:59:32,-500,NaN,Watching inception. Joseph gordon-levitt. Oh y...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1,30776922921246720,30776922921246720,1.296173e+12,2011-01-27 18:59:24,-500,NaN,Ken Watanabe &lt;---Reason to see Inception mo...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,2,30776462000791552,30776462000791552,1.296173e+12,2011-01-27 18:57:34,-500,NaN,#NW Inception! yh yh yh.. stale girl.. waheva....,en,['nw'],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,3,30776412738682880,30776412738682880,1.296173e+12,2011-01-27 18:57:23,-500,NaN,"'Black Swan', 'The Social Network', 'Inception...",en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,4,30776381751173121,30776381751173121,1.296173e+12,2011-01-27 18:57:15,-500,NaN,I just voted for Inception for #BestScreenplay...,en,['bestscreenplay'],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [10]:
df_1.columns

Index(['Unnamed: 0', 'id', 'conversation_id', 'created_at', 'date', 'timezone',
       'place', 'tweet', 'language', 'hashtags', 'cashtags', 'user_id',
       'user_id_str', 'username', 'name', 'day', 'hour', 'link', 'urls',
       'photos', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies',
       'nretweets', 'quote_url', 'search', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [11]:
df_1.drop(columns=['Unnamed: 0', 'id', 'conversation_id', 'created_at', 'timezone',
       'place', 'cashtags', 'user_id', 'user_id_str', 'name',
       'day', 'hour', 'urls', 'photos', 'video', 'thumbnail', 'quote_url', 
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 
       'reply_to', 'translate', 'trans_src', 'trans_dest'], axis=1, inplace = True)

In [12]:
df_1.head()

,date,tweet,language,hashtags,username,link,retweet,nlikes,nreplies,nretweets,retweet_date
0,2011-01-27 18:59:32,Watching inception. Joseph gordon-levitt. Oh y...,en,[],EmilyNMurray,https://twitter.com/EmilyNMurray/status/307769...,False,0,0,0,NaN
1,2011-01-27 18:59:24,Ken Watanabe &lt;---Reason to see Inception mo...,en,[],LisaAsanuma,https://twitter.com/LisaAsanuma/status/3077692...,False,0,0,1,NaN
2,2011-01-27 18:57:34,#NW Inception! yh yh yh.. stale girl.. waheva....,en,['nw'],MsAllurer,https://twitter.com/MsAllurer/status/307764620...,False,0,0,0,NaN
3,2011-01-27 18:57:23,"'Black Swan', 'The Social Network', 'Inception...",en,[],AlexaChill,https://twitter.com/AlexaChill/status/30776412...,False,0,0,0,NaN
4,2011-01-27 18:57:15,I just voted for Inception for #BestScreenplay...,en,['bestscreenplay'],MarioCayetano,https://twitter.com/MarioCayetano/status/30776...,False,0,0,0,NaN


In [14]:
df_1['date'] = pd.to_datetime(df_1['date'])
df_1.head()

,date,tweet,language,hashtags,username,link,retweet,nlikes,nreplies,nretweets,retweet_date
0,2011-01-27 18:59:32,Watching inception. Joseph gordon-levitt. Oh y...,en,[],EmilyNMurray,https://twitter.com/EmilyNMurray/status/307769...,False,0,0,0,NaN
1,2011-01-27 18:59:24,Ken Watanabe &lt;---Reason to see Inception mo...,en,[],LisaAsanuma,https://twitter.com/LisaAsanuma/status/3077692...,False,0,0,1,NaN
2,2011-01-27 18:57:34,#NW Inception! yh yh yh.. stale girl.. waheva....,en,['nw'],MsAllurer,https://twitter.com/MsAllurer/status/307764620...,False,0,0,0,NaN
3,2011-01-27 18:57:23,"'Black Swan', 'The Social Network', 'Inception...",en,[],AlexaChill,https://twitter.com/AlexaChill/status/30776412...,False,0,0,0,NaN
4,2011-01-27 18:57:15,I just voted for Inception for #BestScreenplay...,en,['bestscreenplay'],MarioCayetano,https://twitter.com/MarioCayetano/status/30776...,False,0,0,0,NaN


In [5]:
vader = SentimentIntensityAnalyzer()

In [7]:
df_1['vader_score'] = df_1['tweet'].apply(lambda x: vader.polarity_scores(x))
df_1['vader_score']

0       {'neg': 0.0, 'neu': 0.49, 'pos': 0.51, 'compou...
1       {'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'comp...
2       {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
3       {'neg': 0.0, 'neu': 0.683, 'pos': 0.317, 'comp...
4       {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
                              ...                        
5317    {'neg': 0.174, 'neu': 0.457, 'pos': 0.369, 'co...
5318    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
5319    {'neg': 0.195, 'neu': 0.565, 'pos': 0.24, 'com...
5320    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
5321    {'neg': 0.216, 'neu': 0.455, 'pos': 0.33, 'com...
Name: vader_score, Length: 5322, dtype: object

In [6]:
df_1['vader_score'] = df_1['tweet'].apply(lambda x: vader.polarity_scores(x)['compound'])
df_1['vader_score'].head()

0    0.6369
1    0.4019
2    0.0000
3    0.7579
4    0.0000
Name: vader_score, dtype: float64